# Project Idea:
### I plan to use NLP to analyze the responses given by cities around the world to a questionaire pertaining to environmental and social endeavors. The questionaire is distributed by CDP (formely known as Carbon Disclosure Project). Questionaires are located '../data/raw_data/Cities/Cities Questionaires 
    

In [10]:
#libraries
import pandas as pd
import numpy as np

## Disclosure Documents

#### importing Data

In [11]:
#Import Cities Data
cities_2018_disclosure = pd.read_csv('../data/raw_data/Cities/Cities Disclosing/2018_Cities_Disclosing_to_CDP.csv')
cities_2019_disclosure = pd.read_csv('../data/raw_data/Cities/Cities Disclosing/2019_Cities_Disclosing_to_CDP.csv')
cities_2020_disclosure = pd.read_csv('../data/raw_data/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv')

In [12]:
#combine files
cities_disclosure=[cities_2018_disclosure,cities_2019_disclosure,cities_2020_disclosure]
cities_disclosure= pd.concat(cities_disclosure).sort_values('Account Number').reset_index(drop=True)

#Adjust column names
cities_disclosure.columns=['Reporting_Year','Account_Num','Org','City','Country','CDP_Reg','Reporting_Auth',\
                          'Access','First_Time_Disc','Pop','Pop_Year','City_Loc','Last_Update']
cities_disclosure.head()

#cities_disclosure.shape (20168,13)

,Reporting_Year,Account_Num,Org,City,Country,CDP_Reg,Reporting_Auth,Access,First_Time_Disc,Pop,Pop_Year,City_Loc,Last_Update
0,2019,1093,City of Atlanta,Atlanta,United States of America,North America,"CDP Cities,ICLEI - Local Governments for Susta...",public,No,450182.0,2017,POINT (-84.388 33.749),2020-06-25T04:52:51.327
1,2018,1093,City of Atlanta,Atlanta,United States of America,North America,CDP Cities,public,No,450182.0,2017,POINT (-84.388 33.749),2020-06-25T04:52:49.050
2,2018,1184,City of Austin,Austin,United States of America,North America,"C40,CDP Cities",public,No,1265974.0,2017,POINT (-97.7431 30.2672),2020-06-25T04:52:49.050
3,2020,1184,City of Austin,Austin,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,964254.0,2019,POINT (-97.7431 30.2672),2020-09-07T09:45:19.793
4,2019,1184,City of Austin,Austin,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,1227000.0,2017,POINT (-97.7431 30.2672),2020-06-25T04:52:51.327


In [13]:
len(cities_disclosure.Country.unique())

96

### Dealing with Nulls

##### City

In [14]:
#Lets look at nulls
cities_disclosure.isnull().sum()

Reporting_Year       0
Account_Num          0
Org                  0
City               352
Country              0
CDP_Reg              0
Reporting_Auth       0
Access               0
First_Time_Disc      0
Pop                284
Pop_Year             0
City_Loc           438
Last_Update          0
dtype: int64

The dataset contains data from both inside and outside of the US. It may be more common for countries to not list a city (since they may not use cities). I'll set non-US nulls to "Not Listed" and remove US nulls if the number of missing data is less than 5%.

In [15]:
#Nulls from outside the US
cities_disclosure.loc[(cities_disclosure.City.isnull())&\
                      (cities_disclosure.Country!='United States of America'),'City']='Not Listed'

In [16]:
#Nulls from inside the US
num = cities_disclosure.loc[(cities_disclosure.City.isnull())&\
                      (cities_disclosure.Country=='United States of America')].shape[0]
denom = cities_disclosure['City'].shape[0]

print('Total percentage of missing nulls are {}%'.format(round((num/denom)*100,2)))

Total percentage of missing nulls are 2.03%


##### Population

Find an example where we have populations for some dates but not others

In [17]:
#isolate an example
cities_disclosure.loc[cities_disclosure['Account_Num']==10894]

,Reporting_Year,Account_Num,Org,City,Country,CDP_Reg,Reporting_Auth,Access,First_Time_Disc,Pop,Pop_Year,City_Loc,Last_Update
33,2018,10894,City of Los Angeles,Los Angeles,United States of America,North America,"C40,CDP Cities",non public,No,4030904.0,2016,POINT (-118.244 34.0522),2020-06-25T04:52:49.050
34,2020,10894,City of Los Angeles,Los Angeles,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,4021488.0,2018,POINT (-118.244 34.0522),2020-09-07T09:45:19.793
35,2019,10894,City of Los Angeles,Los Angeles,United States of America,North America,"C40,CDP Cities,ICLEI - Local Governments for S...",public,No,NaN,0,POINT (-118.244 34.0522),2020-06-25T04:52:51.327


In [18]:
#fill the nulls with the mean values
cities_disclosure['Pop'] = \
cities_disclosure.groupby(['Account_Num'],sort=False)['Pop'].apply(lambda x: x.fillna(x.mean()))

In [19]:
#verify that our solution is correct
assert cities_disclosure.loc[35,'Pop']==4026196.0

##### Population Year

Replace pop_year with average year (to match us taking the average population value)

In [20]:
#replace 0s with nulls
cities_disclosure['Pop_Year'] = \
cities_disclosure.groupby(['Account_Num'])['Pop_Year'].apply(lambda x: x.replace(0,np.nan))


#replace nulls with averag value

cities_disclosure['Pop_Year'] = \
cities_disclosure.groupby(['Account_Num'])['Pop_Year'].apply(lambda x: x.fillna(x.mean()))

In [21]:
assert cities_disclosure.loc[35,'Pop_Year']==2017

In [22]:
cities_disclosure.Pop_Year.unique()

array([2017. , 2019. , 2018. , 2015. , 2016. , 2020. , 2012. , 2011. ,
       2017.5, 2013. , 2010. , 2014. , 2016.5,    nan, 2018.5, 2013.5,
          7. , 1013. , 2007. ,  214. , 2005. , 2012.5, 2019.5,  216. ,
        217. ,   19. , 2003. , 2014.5, 2009. , 2008. ,  218. ,  219. ,
         21. , 1537. ,  211. , 2002. ])

From the output, we still have some issues. We need to cirlce back to these

## Responses

In [25]:
#pull in data and merge like above
Cities_2018_Responses = pd.read_csv('../data/raw_data/Cities/Cities Responses/2018_Full_Cities_Dataset.csv')
Cities_2019_Responses = pd.read_csv('../data/raw_data/Cities/Cities Responses/2019_Full_Cities_Dataset.csv')
Cities_2020_Responses = pd.read_csv('../data/raw_data/Cities/Cities Responses/2020_Full_Cities_Dataset.csv')

Cities_Responses=pd.concat([Cities_2018_Responses,Cities_2019_Responses,Cities_2020_Responses])\
.sort_values('Account Number').reset_index(drop=True)
Cities_Responses.head()

,Questionnaire,Year Reported to CDP,Account Number,Organization,Country,CDP Region,Parent Section,Section,Question Number,Question Name,Column Number,Column Name,Row Number,Row Name,Response Answer,Comments,File Name,Last update
0,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Opportunities,Opportunities,6.2,"List any emission reduction, adaptation, water...",5,Project description,2,NaN,Renew Atlanta is an infrastructure improvement...,NaN,NaN,24/06/2020 05:30:36 AM
1,CDP Cities 2018,2018,1093,City of Atlanta,United States of America,North America,City Wide Emissions,GHG Emissions Data,7.5,Please attach your city-wide inventory in the ...,4,Comment on level of confidence,1,NaN,We were unable to retrieve natural gas data fr...,NaN,NaN,24/06/2020 05:28:18 AM
2,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,12,Please indicate to which sector(s) the target ...,1,NaN,Public facility,NaN,NaN,24/06/2020 05:30:36 AM
3,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,9,Target year absolute emissions (metric tonnes ...,1,NaN,NaN,NaN,NaN,24/06/2020 05:30:36 AM
4,Cities 2019,2019,1093,City of Atlanta,United States of America,North America,Climate Hazards & Vulnerability,Climate Hazards,2.2,Please identify and describe the factors that ...,2,Support / Challenge,4,NaN,Challenge,NaN,NaN,24/06/2020 05:30:36 AM


In [26]:
#drop the columns that are also present in the disclosure file
Cities_Responses.drop(columns=['Questionnaire','Organization','Country','CDP Region',\
                               'File Name','Last update','Comments'],inplace=True)

In [27]:
#rename columns
Cities_Responses.columns=['Year','Account_Num','Parent_Sect','Sect','Q_Num','Q_Name',\
                          'Col_Num','Col_Name','Row_Num','Row_Name','Answer']
Cities_Responses.head()

,Year,Account_Num,Parent_Sect,Sect,Q_Num,Q_Name,Col_Num,Col_Name,Row_Num,Row_Name,Answer
0,2019,1093,Opportunities,Opportunities,6.2,"List any emission reduction, adaptation, water...",5,Project description,2,NaN,Renew Atlanta is an infrastructure improvement...
1,2018,1093,City Wide Emissions,GHG Emissions Data,7.5,Please attach your city-wide inventory in the ...,4,Comment on level of confidence,1,NaN,We were unable to retrieve natural gas data fr...
2,2019,1093,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,12,Please indicate to which sector(s) the target ...,1,NaN,Public facility
3,2019,1093,Emissions Reduction,Mitigation Target setting,5.0a,Please provide details of your total city-wide...,9,Target year absolute emissions (metric tonnes ...,1,NaN,NaN
4,2019,1093,Climate Hazards & Vulnerability,Climate Hazards,2.2,Please identify and describe the factors that ...,2,Support / Challenge,4,NaN,Challenge


# THIS TOOK ME FOREVER TO FIGURE OUT!

the raw data actually provides an additional records in instances where the respondee could provide multiple answers without identifying them via a new unique identifier(a different row_num tag). Therefore I'll group responses together on like IDs and merge the answers together via a join

In [28]:
Cities_Responses.Answer = Cities_Responses.Answer.astype('str')

In [32]:
report_questions = Cities_Responses.groupby(['Year',
                          'Account_Num',
                          'Parent_Sect',
                          'Sect',
                          'Q_Num',
                          'Col_Num',
                          'Row_Num'], as_index=False).size()
report_questions.query("size>1")

,Year,Account_Num,Parent_Sect,Sect,Q_Num,Col_Num,Row_Num,size
104,2018,1093,Climate Hazards,Climate Hazards,2.2a,9,1,3
105,2018,1093,Climate Hazards,Climate Hazards,2.2a,9,2,3
106,2018,1093,Climate Hazards,Climate Hazards,2.2a,9,3,3
107,2018,1093,Climate Hazards,Climate Hazards,2.2a,9,4,3
108,2018,1093,Climate Hazards,Climate Hazards,2.2a,9,5,3
...,...,...,...,...,...,...,...,...
1140759,2020,848478,Climate Hazards and Vulnerability,Climate Hazards,2.1,6,1,2
1140760,2020,848478,Climate Hazards and Vulnerability,Climate Hazards,2.1,6,2,2
1140761,2020,848478,Climate Hazards and Vulnerability,Climate Hazards,2.1,7,1,2
1140815,2020,848478,Emissions Reduction,Mitigation Actions,5.4,9,1,4


In [34]:
Cities_Responses.query("Account_Num ==1093 & Sect=='Climate Hazards' & Q_Num =='2.2a' & Col_Num ==9 & Row_Num ==1")

,Year,Account_Num,Parent_Sect,Sect,Q_Num,Q_Name,Col_Num,Col_Name,Row_Num,Row_Name,Answer
62,2018,1093,Climate Hazards,Climate Hazards,2.2a,Please list the most significant climate hazar...,9,Top three assets/ services affected,1,NaN,Emergency management
241,2018,1093,Climate Hazards,Climate Hazards,2.2a,Please list the most significant climate hazar...,9,Top three assets/ services affected,1,NaN,Food & agriculture
747,2018,1093,Climate Hazards,Climate Hazards,2.2a,Please list the most significant climate hazar...,9,Top three assets/ services affected,1,NaN,Water supply & sanitation


In [45]:
Cities_Responses_Grouped = Cities_Responses.groupby(['Year',
                          'Account_Num',
                          'Parent_Sect',
                          'Sect',
                          'Q_Num',
                          'Col_Num',
                          'Row_Num'], as_index=False)['Answer'].apply(', '.join)       

In [46]:
Cities_Responses_Grouped.query("Account_Num ==1093 & Sect=='Climate Hazards' & Q_Num =='2.2a' & Col_Num ==9 & Row_Num ==1")

,Year,Account_Num,Parent_Sect,Sect,Q_Num,Col_Num,Row_Num,Answer
104,2018,1093,Climate Hazards,Climate Hazards,2.2a,9,1,"Emergency management, Food & agriculture, Wate..."


## Constructing Relational Datasets

In [47]:
#Create table of unique questions from the questionaire
Questionaire_Questions =Cities_Responses[['Year','Parent_Sect','Sect','Q_Num',\
                                          'Q_Name','Col_Num','Col_Name','Row_Num','Row_Name']].drop_duplicates()
#reset index
Questionaire_Questions=Questionaire_Questions.\
        sort_values(['Parent_Sect','Sect','Q_Num','Col_Num','Row_Num','Year'])

#we now need to drop duplicate rows 
Questionaire_Questions.drop_duplicates(inplace=True)

#add a unique ID for each row via indexing
Questionaire_Questions = Questionaire_Questions.reset_index(drop=True)

#Change index name to Question ID
Questionaire_Questions.index.name="Question_ID"
Questionaire_Questions.reset_index(inplace=True)

#Head
Questionaire_Questions.head()

,Question_ID,Year,Parent_Sect,Sect,Q_Num,Q_Name,Col_Num,Col_Name,Row_Num,Row_Name
0,0,2019,Adaptation,Adaptation Actions,3.0,Please describe the main actions you are takin...,1,Climate hazards,0,NaN
1,1,2020,Adaptation,Adaptation Actions,3.0,Please describe the main actions you are takin...,1,Climate hazards,0,NaN
2,2,2019,Adaptation,Adaptation Actions,3.0,Please describe the main actions you are takin...,1,Climate hazards,1,NaN
3,3,2020,Adaptation,Adaptation Actions,3.0,Please describe the main actions you are takin...,1,Climate hazards,1,NaN
4,4,2019,Adaptation,Adaptation Actions,3.0,Please describe the main actions you are takin...,1,Climate hazards,2,NaN


In [48]:
#create our new FACT_DF 
    #that contains Account_Num(Key from disclosure DF) and 
    #Question_Id (Key from questionare_question df)
    
df = Cities_Responses_Grouped.merge(Questionaire_Questions,on=['Parent_Sect','Sect','Year','Q_Num','Col_Num','Row_Num'])

#subset
df = df[['Year','Account_Num','Question_ID','Answer']]

In [49]:
df.columns.to_list()

['Year', 'Account_Num', 'Question_ID', 'Answer']

Lets test to make sure that the question_ID corresponds to the correct Answer.

In [50]:
#Create a function to test that our merges line up properly. 
def test_merge_Q_id(account_Num,Question_ID,answer):
    '''
    This is to make sure that I lined up the Questionare_Questions DF to the Cities_Response DataFrame.
    Enter an account_Num, Question_ID, and the answer that you are expecting
    '''
    
    #isolate from the questionaire df
    qq=Questionaire_Questions.loc[Questionaire_Questions.Question_ID==Question_ID] 
    
    #isolate from the responses df
    cr=Cities_Responses.loc[Cities_Responses['Account_Num']==account_Num]\
    [['Answer','Year','Q_Num','Q_Name','Col_Num','Col_Name','Row_Num','Row_Name']]
    
    #make sure the data matches 
    result = cr.loc[(cr.Year.isin(qq.Year))&\
                    (cr.Q_Num.isin(qq.Q_Num))&\
                    (cr.Col_Num.isin(qq.Col_Num))&\
                    (cr.Row_Num.isin(qq.Row_Num)),'Answer']
    
    #make sure the answers match
    assert answer == result.values


In [51]:
#test the function by pluging in an account_num, quesitno_id and the expected answer
test_merge_Q_id(56276,5926,'250')
test_merge_Q_id(31182,18414,'EV Charging Infrastructure. \
The Port of San Francisco is currently seeking an outside \
partner to install EV charging infrastructure on Port property \
and make it available for public charging.')

All good! No errors

## Cleaning up the df

In [52]:
df.dtypes

Year            int64
Account_Num     int64
Question_ID     int64
Answer         object
dtype: object

In [53]:
#Check for nulls
df.isnull().sum()

Year           0
Account_Num    0
Question_ID    0
Answer         0
dtype: int64

In [54]:
df.loc[df.Answer.isna()]

,Year,Account_Num,Question_ID,Answer


we can remove these from our df since they are blanks

In [55]:
df.dropna(inplace=True)

In [56]:
df.head()

,Year,Account_Num,Question_ID,Answer
0,2018,1093,1655,Rain storm
1,2018,1184,1655,nan
2,2018,1499,1655,Rain storm
3,2018,3203,1655,Flash/surface flood
4,2018,3417,1655,Coastal flood


## Inspect Geospatial Analysis

In [57]:
#review values
cities_disclosure.City_Loc.unique()[:10]

array(['POINT (-84.388 33.749)', 'POINT (-97.7431 30.2672)',
       'POINT (2.1775 41.3823)', nan, 'POINT (-73.2121 44.4759)',
       'POINT (-87.6298 41.8781)', 'POINT (-74.0059 40.7128)',
       'POINT (-0.07868 51.5048)', 'POINT (18.0686 59.3293)',
       'POINT (24.941 60.1733)'], dtype=object)

In [58]:
#rename the geo data to match convention
cities_disclosure.rename(columns={'City_Loc':'geometry'},inplace=True)

In [59]:
#remove null geopoints from the dataset
CD_df = cities_disclosure.dropna()

In [60]:
#add Lat Long coords
CD_df['lat']=CD_df.geometry.apply(lambda x: x.split(' ')[2].strip(')'))
CD_df['long']=CD_df.geometry.apply(lambda x: x.split(' ')[1].strip('('))

/var/folders/r8/kdff6gm9553_wdcqb3x5r7vh0000gq/T/ipykernel_8682/742568004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CD_df['lat']=CD_df.geometry.apply(lambda x: x.split(' ')[2].strip(')'))
/var/folders/r8/kdff6gm9553_wdcqb3x5r7vh0000gq/T/ipykernel_8682/742568004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CD_df['long']=CD_df.geometry.apply(lambda x: x.split(' ')[1].strip('('))


In [61]:
### Saving Data to interim folder
CD_df.to_csv('../data/interim_data/dim_cd_df.csv')
df.to_csv('../data/interim_data/FACT_df.csv')
Questionaire_Questions.to_csv('../data/interim_data/dim_qq_df.csv')